### Marshmallow

https://marshmallow.readthedocs.io/en/stable/

Marshmallow is used for validation as well as serialization and deserialization of objects

But it goes beyond just JSON too!

In [20]:
class Person:
    def __init__(self, first_name, last_name, dob, height):
        self.first_name = first_name
        self.last_name = last_name
        self.dob = dob
        self.height = height
        
    def __repr__(self):
        return f'Person({self.first_name}, {self.last_name}, {self.dob}, {self.height})'

In [6]:
from datetime import date

In [21]:
p1 = Person('John', 'Cleese', date(1939, 10, 27), 182)

In [22]:
p1

Person(John, Cleese, 1939-10-27, 182)

In [10]:
pip install marshmallow

Note: you may need to restart the kernel to use updated packages.


In [31]:
from marshmallow import Schema, fields

In [32]:
class PersonSchema(Schema):
    first_name = fields.Str()
    last_name = fields.Str()
    dob = fields.Date()
    height = fields.Int()

In [33]:
person_schema = PersonSchema()

In [34]:
person_schema.dump(p1)

{'first_name': 'John',
 'height': 182,
 'last_name': 'Cleese',
 'dob': '1939-10-27'}

In [35]:
data = person_schema.dump(p1)

In [36]:
data

{'first_name': 'John',
 'height': 182,
 'last_name': 'Cleese',
 'dob': '1939-10-27'}

In [37]:
print(type(p1), p1)
print(type(data), data)


<class '__main__.Person'> Person(John, Cleese, 1939-10-27, 182)
<class 'dict'> {'first_name': 'John', 'height': 182, 'last_name': 'Cleese', 'dob': '1939-10-27'}


In [38]:
type(data['height'])

int

In [39]:
import json

In [40]:
json.dumps(data)

'{"first_name": "John", "height": 182, "last_name": "Cleese", "dob": "1939-10-27"}'

In [41]:
data = person_schema.dumps(p1)

In [42]:
data

'{"first_name": "John", "height": 182, "last_name": "Cleese", "dob": "1939-10-27"}'

In [43]:
type(data)

str

In [44]:
from collections import namedtuple

In [45]:
PT = namedtuple('PT', 'first_name, last_name, dob, height')

In [47]:
p2 = PT('Eric', 'Idle', date(1943, 3, 29), 178)

In [48]:
p2

PT(first_name='Eric', last_name='Idle', dob=datetime.date(1943, 3, 29), height=178)

In [50]:
person_schema.dumps(p2)

'{"first_name": "Eric", "height": 178, "last_name": "Idle", "dob": "1943-03-29"}'

In [52]:
PT2 = namedtuple('PT2', 'first_name, last_name, age')
p3 = PT2('Mihcael', 'Palin', 75)

In [53]:
person_schema.dumps(p3)

'{"first_name": "Mihcael", "last_name": "Palin"}'

In [54]:
person_partial = PersonSchema(only=('first_name', 'last_name'))

In [55]:
person_schema.dumps(p1)

'{"first_name": "John", "height": 182, "last_name": "Cleese", "dob": "1939-10-27"}'

In [56]:
person_partial.dumps(p1)

'{"first_name": "John", "last_name": "Cleese"}'

In [57]:
person_partial = PersonSchema(exclude=['dob'])

In [58]:
person_partial.dumps(p1)

'{"first_name": "John", "height": 182, "last_name": "Cleese"}'

In [59]:
class PersonSchema(Schema):
    first_name = fields.Str()
    last_name = fields.Str()
    dob = fields.Date()
    height = fields.Int()

In [60]:
p4 = Person(100, None, 200, 'abc')

In [61]:
person_schema.dumps(p4)

ValueError: invalid literal for int() with base 10: 'abc'

In [62]:
class Movie:
    def __init__(self, title, year, actors):
        self.title = title
        self.year = year
        self.actors = actors

In [63]:
class MovieSchema(Schema):
    title = fields.Str()
    year = fields.Int()
    actors = fields.Nested(PersonSchema, many=True)

In [64]:
p1, p2

(Person(John, Cleese, 1939-10-27, 182),
 PT(first_name='Eric', last_name='Idle', dob=datetime.date(1943, 3, 29), height=178))

In [66]:
parrot = Movie('Parrot Sketch', 1989, [p1, PT('Michael', 'Palin', date(1943, 5, 5), 177)])

In [67]:
parrot

In [68]:
MovieSchema().dumps(parrot)

'{"actors": [{"first_name": "John", "height": 182, "last_name": "Cleese", "dob": "1939-10-27"}, {"first_name": "Michael", "height": 177, "last_name": "Palin", "dob": "1943-05-05"}], "year": 1989, "title": "Parrot Sketch"}'

In [76]:
class PersonSchema(Schema):
    first_name = fields.Str()
    last_name = fields.Str()
    dob = fields.Date()
    height = fields.Int()

In [77]:
person_schema = PersonSchema()

In [78]:
person_schema.load(dict(first_name='John',
                       last_name='Cleese',
                       dob='1939-10-27',
                       height=178
                       ))

{'first_name': 'John',
 'height': 178,
 'last_name': 'Cleese',
 'dob': datetime.date(1939, 10, 27)}

Notice dob is now a datetime object not a string

In [79]:
from marshmallow import post_load

In [82]:
class PersonSchema(Schema):
    first_name = fields.Str()
    last_name = fields.Str()
    dob = fields.Date()
    height = fields.Int()
    
    @post_load
    def make_person(self, data, **kwargs):
        return Person(**data)

In [83]:
PersonSchema().load(dict(first_name='John',
                       last_name='Cleese',
                       dob='1939-10-27',
                       height=178
                       ))

Person(John, Cleese, 1939-10-27, 178)

In [84]:
class MovieSchema(Schema):
    title = fields.Str()
    year = fields.Int()
    actors = fields.Nested(PersonSchema, many=True)
    
    @post_load
    def make_movie(self, data, **kwargs):
        return Movie(**data)

In [85]:
movie_schema = MovieSchema()
person_schema = PersonSchema()

In [88]:
json_data = '''
{
    "actors": [
        {"first_name": "John", "last_name": "Cleese", "dob": "1939-10-27", "height": 183},
        {"first_name": "Michael", "last_name": "Palin", "dob": "1943-05-05", "height": 178}
        ],
        "title": "Parrot Sketch",
        "year": 1989
}
'''

In [89]:
movie = movie_schema.loads(json_data)

In [90]:
movie

In [91]:
movie.title

'Parrot Sketch'

In [92]:
movie.actors

[Person(John, Cleese, 1939-10-27, 183),
 Person(Michael, Palin, 1943-05-05, 178)]